# Training data

In [2]:
import pickle
import numpy as np
import random
from operator import add

In [3]:
f = open ('./char_embedding.pkl', 'rb')
dict_char = pickle.load(f)
f.close()

f = open ('./dataset/item_index_withEN.pkl', 'rb')
item_index = pickle.load(f)
f.close()

f = open ('./dataset/name_index_withEN.pkl', 'rb')
user_index = pickle.load(f)
f.close()

In [4]:
rating_mat = np.load('dataset/user_item_withEN_mat.npy')

In [5]:
print("dict_char", len(dict_char))
print("user_index", len(user_index))
print("item_index", len(item_index))
print("rating_mat", rating_mat.shape)

dict_char 21592
user_index 15251
item_index 32
rating_mat (15251, 32)


In [6]:
#dict_char['沈']
#user_list.remove('')
k, v = random.choice(list(dict_char.items()))
print(type(v))
#print(user_list)

<class 'numpy.ndarray'>


In [8]:
user_list = list(user_index.keys())
print(len(user_list))
user_mat = []
user_emb = []
count = 0

for user in user_list:
    temp = []
    for char in user:
        try:
            temp.append(dict_char[char])
        except:
            count += 1
            k, v = random.choice(list(dict_char.items()))
            temp.append(v)
    #print(len(temp))
    if len(temp) == 2:
        user_emb = np.mean([temp[0], temp[1]], axis=0)
    elif len(temp) == 3:
        user_emb = np.mean([temp[0], temp[1], temp[2]], axis=0)
    elif len(temp) == 4:
        user_emb = np.mean([temp[0], temp[1], temp[2], temp[-1]], axis=0)
    else:
        print("index", user_list.index(user))
        #rating_mat = np.delete(rating_mat, [user_list.index(user)], axis=0)
        user_list.remove(user)
        del user_index[user]
        print(len(temp))
    
    user_mat.append(user_emb)

user_mat = np.array(user_mat)
print("missing char", count)
print("user_mat", user_mat.shape)
print("remain", len(user_list))
print("remain", rating_mat.shape)

15250
missing char 190
user_mat (15250, 256)
remain 15250
remain (15250, 32)


In [9]:
#item_shape = (256, 32)
#item_mat = np.random.rand(32, 256)
print(user_mat)

[[-0.10925939  0.10057906 -0.36214566 ...  0.22985218  0.11172808
  -0.2572367 ]
 [ 0.13928922 -0.27870765 -0.24850345 ...  0.29933026 -0.26288828
  -0.47202894]
 [-0.04528208  0.270188   -0.06590408 ...  0.58950824  0.3769443
   0.06053051]
 ...
 [-0.3130688  -0.13624458  0.14155173 ...  0.67227966  0.05476775
  -0.7716191 ]
 [ 0.3513397   0.02466246 -0.91061765 ...  0.09871349  0.07908598
  -0.59299964]
 [-0.5004844   0.39976573  0.15475018 ...  0.16018112  0.06941525
  -0.19348325]]


In [10]:
num_user = 15250
num_item = 32

# Model

In [11]:
from keras import Model
import keras.backend as K
from keras.layers import Embedding,Reshape,Input,Dot, Dense, dot, Lambda
K.clear_session()

/Users/Solitude6060/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/Solitude6060/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [12]:
# def create_model(num_user, num_item, k):
#     input_user = Input(shape=(256,),dtype="float32")
#     #model_uer = Embedding(num_user+1,k,input_length = 1)(input_uer)
#     model_user = Dense(k, activation='relu')(input_user)
#     model_user = Reshape((k,))(model_user)
    
#     input_item = Input(shape=(32,),dtype="float32")
#     model_item  = Embedding(num_item+1,k,input_length = 32)(input_item)
#     model_item = Dense(256, activation='relu')(model_item)
#     model_item = Reshape((256,))(model_item)
    
#     out = Dot(1)([model_user,model_item])
#     model = Model(inputs=[input_user,model_item], outputs=out)
#     model.compile(loss='mse', optimizer='Adam')
#     model.summary()
#     return model

In [136]:
def Recmand_model(num_user,num_item,k):
    input_uer = Input(shape=(256, ),dtype="float32")
    model_uer = Dense(k, activation='relu')(input_uer)
    
    
    input_item = Input(shape=(1,), dtype="float32")
    model_item  = Embedding(num_item, k, input_length = 1)(input_item)
    model_item = Reshape((k,))(model_item)
    
    out = Lambda(lambda x: K.sum(x[0]*x[1], axis=-1, keepdims=True))([model_uer, model_item])
    model = Model(inputs=[input_uer, input_item], outputs=out)
    model.compile(loss='mse', optimizer='Adam')
    model.summary()
    return model

In [137]:
model = Recmand_model(num_user, num_item, 256)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 256)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 256)       8192        input_4[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 256)          65792       input_3[0][0]                    
__________________________________________________________________________________________________
reshape_2 

In [138]:
# batch_size = 10
test_user = np.random.random((10, 256))
test_item = np.random.randint(0,32, (10))

In [139]:
print(test_user)

[[0.99612304 0.29585489 0.62131548 ... 0.35031416 0.61339827 0.94748822]
 [0.06105254 0.09523447 0.47658897 ... 0.25967905 0.67530497 0.76091635]
 [0.15442327 0.41536417 0.65538094 ... 0.56931575 0.65029778 0.69949845]
 ...
 [0.00454594 0.37337829 0.56416679 ... 0.92785256 0.66219697 0.24483472]
 [0.40131438 0.43482024 0.76598484 ... 0.98168576 0.02519858 0.79735109]
 [0.00324566 0.77789904 0.57962208 ... 0.60030407 0.01615584 0.95639749]]


In [140]:
print(test_item)

[12 19 27 28 30 12  2  6  7  5]


In [141]:
# test predict
#model.predict([np.random.random((10, 256)), np.random.randint(0,32, (10))])

In [142]:
print(user_mat)

[[-0.10925939  0.10057906 -0.36214566 ...  0.22985218  0.11172808
  -0.2572367 ]
 [ 0.13928922 -0.27870765 -0.24850345 ...  0.29933026 -0.26288828
  -0.47202894]
 [-0.04528208  0.270188   -0.06590408 ...  0.58950824  0.3769443
   0.06053051]
 ...
 [-0.3130688  -0.13624458  0.14155173 ...  0.67227966  0.05476775
  -0.7716191 ]
 [ 0.3513397   0.02466246 -0.91061765 ...  0.09871349  0.07908598
  -0.59299964]
 [-0.5004844   0.39976573  0.15475018 ...  0.16018112  0.06941525
  -0.19348325]]


In [143]:
user_index = 0
item_index = 0
train_x = []
train_user = []
train_item = []

for u in rating_mat:
    for i in u:
        if i == 1:
            # train_x.append([user_mat[user_index], [item_index]])
            u_list = user_mat[user_index].tolist()
            train_user.append(u_list)
            #print(user_mat[user_index].tolist())
            train_item.append(item_index)
        item_index += 1
    user_index += 1
    item_index = 0
train_x = [np.array(train_user), np.array(train_item)]

In [144]:
#train_x = np.array(train_x)
print(train_x)
#print(train_x)
train_y = [1] * 39900
train = [1] * 10
#print(train_y)

[array([[-0.10925939,  0.10057906, -0.36214566, ...,  0.22985218,
         0.11172808, -0.25723669],
       [-0.10925939,  0.10057906, -0.36214566, ...,  0.22985218,
         0.11172808, -0.25723669],
       [-0.10925939,  0.10057906, -0.36214566, ...,  0.22985218,
         0.11172808, -0.25723669],
       ...,
       [-0.31306881, -0.13624458,  0.14155173, ...,  0.67227966,
         0.05476775, -0.77161908],
       [ 0.3513397 ,  0.02466246, -0.91061765, ...,  0.09871349,
         0.07908598, -0.59299964],
       [-0.50048441,  0.39976573,  0.15475018, ...,  0.16018112,
         0.06941525, -0.19348325]]), array([ 6, 12, 13, ...,  0,  0,  0])]


In [ ]:
model.fit(train_x, train_y, batch_size = 64, epochs =3000, verbose=1, callbacks=None, validation_split=0.2, shuffle=True)

Train on 31920 samples, validate on 7980 samples
Epoch 1/3000
31920/31920 [==============================] - 2s 54us/step - loss: 4.2361e-04 - val_loss: 0.1686
Epoch 2/3000
31920/31920 [==============================] - 2s 51us/step - loss: 5.1675e-04 - val_loss: 0.1717
Epoch 3/3000
31920/31920 [==============================] - 1s 41us/step - loss: 3.9676e-04 - val_loss: 0.1750
Epoch 4/3000
31920/31920 [==============================] - 1s 43us/step - loss: 4.4455e-04 - val_loss: 0.1705
Epoch 5/3000
31920/31920 [==============================] - 1s 42us/step - loss: 3.3296e-04 - val_loss: 0.1735
Epoch 6/3000
31920/31920 [==============================] - 1s 44us/step - loss: 3.0545e-04 - val_loss: 0.1759
Epoch 7/3000
31920/31920 [==============================] - 1s 47us/step - loss: 3.3239e-04 - val_loss: 0.1791
Epoch 8/3000
31920/31920 [==============================] - 1s 42us/step - loss: 2.8270e-04 - val_loss: 0.1773
Epoch 9/3000
31920/31920 [==============================] - 2s 

Epoch 74/3000
31920/31920 [==============================] - 1s 41us/step - loss: 3.8675e-05 - val_loss: 0.2404
Epoch 75/3000
31920/31920 [==============================] - 1s 39us/step - loss: 4.5048e-05 - val_loss: 0.2409
Epoch 76/3000
31920/31920 [==============================] - 1s 38us/step - loss: 3.7526e-05 - val_loss: 0.2420
Epoch 77/3000
31920/31920 [==============================] - 1s 37us/step - loss: 4.3299e-05 - val_loss: 0.2408
Epoch 78/3000
31920/31920 [==============================] - 1s 38us/step - loss: 5.6798e-05 - val_loss: 0.2414
Epoch 79/3000
31920/31920 [==============================] - 1s 38us/step - loss: 1.9229e-05 - val_loss: 0.2415
Epoch 80/3000
31920/31920 [==============================] - 1s 39us/step - loss: 3.1690e-05 - val_loss: 0.2425
Epoch 81/3000
31920/31920 [==============================] - 1s 38us/step - loss: 5.1523e-05 - val_loss: 0.2429
Epoch 82/3000
31920/31920 [==============================] - 1s 42us/step - loss: 3.2066e-05 - val_loss:

31920/31920 [==============================] - 1s 38us/step - loss: 2.8781e-05 - val_loss: 0.2455
Epoch 147/3000
31920/31920 [==============================] - 1s 37us/step - loss: 1.1059e-05 - val_loss: 0.2451
Epoch 148/3000
31920/31920 [==============================] - 1s 38us/step - loss: 1.1281e-05 - val_loss: 0.2453
Epoch 149/3000
31920/31920 [==============================] - 1s 38us/step - loss: 2.8062e-05 - val_loss: 0.2453
Epoch 150/3000
31920/31920 [==============================] - 1s 39us/step - loss: 1.4905e-05 - val_loss: 0.2454
Epoch 151/3000
31920/31920 [==============================] - 1s 37us/step - loss: 1.6784e-05 - val_loss: 0.2453
Epoch 152/3000
31920/31920 [==============================] - 1s 40us/step - loss: 1.8234e-05 - val_loss: 0.2453
Epoch 153/3000
31920/31920 [==============================] - 1s 46us/step - loss: 1.3530e-05 - val_loss: 0.2454
Epoch 154/3000
31920/31920 [==============================] - 1s 42us/step - loss: 1.7941e-05 - val_loss: 0.245

31920/31920 [==============================] - 1s 37us/step - loss: 5.1787e-06 - val_loss: 0.2439
Epoch 219/3000
31920/31920 [==============================] - 1s 37us/step - loss: 1.7154e-05 - val_loss: 0.2436
Epoch 220/3000
31920/31920 [==============================] - 1s 38us/step - loss: 1.3771e-05 - val_loss: 0.2436
Epoch 221/3000
31920/31920 [==============================] - 1s 38us/step - loss: 1.6169e-05 - val_loss: 0.2438
Epoch 222/3000
31920/31920 [==============================] - 1s 38us/step - loss: 1.6538e-05 - val_loss: 0.2439
Epoch 223/3000
31920/31920 [==============================] - 1s 37us/step - loss: 9.6521e-06 - val_loss: 0.2436
Epoch 224/3000
31920/31920 [==============================] - 1s 38us/step - loss: 1.6952e-05 - val_loss: 0.2435
Epoch 225/3000
31920/31920 [==============================] - 1s 38us/step - loss: 1.1106e-05 - val_loss: 0.2434
Epoch 226/3000
31920/31920 [==============================] - 1s 41us/step - loss: 1.5489e-05 - val_loss: 0.243

31920/31920 [==============================] - 1s 38us/step - loss: 4.8613e-06 - val_loss: 0.2407
Epoch 291/3000
31920/31920 [==============================] - 1s 37us/step - loss: 1.5000e-05 - val_loss: 0.2408
Epoch 292/3000
31920/31920 [==============================] - 1s 38us/step - loss: 1.7492e-05 - val_loss: 0.2409
Epoch 293/3000
31920/31920 [==============================] - 1s 38us/step - loss: 1.8058e-06 - val_loss: 0.2408
Epoch 294/3000
31920/31920 [==============================] - 1s 39us/step - loss: 1.2229e-05 - val_loss: 0.2408
Epoch 295/3000
31920/31920 [==============================] - 1s 38us/step - loss: 9.9059e-06 - val_loss: 0.2408
Epoch 296/3000
31920/31920 [==============================] - 1s 38us/step - loss: 1.0871e-05 - val_loss: 0.2407
Epoch 297/3000
31920/31920 [==============================] - 1s 41us/step - loss: 1.2382e-05 - val_loss: 0.2411
Epoch 298/3000
31920/31920 [==============================] - 1s 38us/step - loss: 6.6416e-06 - val_loss: 0.241

...

31920/31920 [==============================] - 1s 38us/step - loss: 6.8807e-06 - val_loss: 0.2294
Epoch 795/3000
31920/31920 [==============================] - 1s 37us/step - loss: 9.0546e-07 - val_loss: 0.2294
Epoch 796/3000
31920/31920 [==============================] - 1s 38us/step - loss: 1.7588e-06 - val_loss: 0.2294
Epoch 797/3000
31920/31920 [==============================] - 1s 41us/step - loss: 7.9466e-06 - val_loss: 0.2294
Epoch 798/3000
31920/31920 [==============================] - 1s 37us/step - loss: 1.3694e-06 - val_loss: 0.2294
Epoch 799/3000
31920/31920 [==============================] - 1s 37us/step - loss: 6.5938e-06 - val_loss: 0.2293
Epoch 800/3000
31920/31920 [==============================] - 1s 37us/step - loss: 1.3433e-06 - val_loss: 0.2294
Epoch 801/3000
31920/31920 [==============================] - 1s 38us/step - loss: 8.3781e-06 - val_loss: 0.2293
Epoch 802/3000
31920/31920 [==============================] - 1s 38us/step - loss: 1.0507e-07 - val_loss: 0.229

31920/31920 [==============================] - 1s 38us/step - loss: 1.7841e-06 - val_loss: 0.2290
Epoch 867/3000
31920/31920 [==============================] - 1s 37us/step - loss: 3.6599e-06 - val_loss: 0.2290
Epoch 868/3000
31920/31920 [==============================] - 1s 37us/step - loss: 6.6000e-06 - val_loss: 0.2290
Epoch 869/3000
31920/31920 [==============================] - 1s 37us/step - loss: 1.0384e-06 - val_loss: 0.2290
Epoch 870/3000
31920/31920 [==============================] - 1s 38us/step - loss: 3.9218e-06 - val_loss: 0.2290
Epoch 871/3000
31920/31920 [==============================] - 1s 37us/step - loss: 3.3549e-06 - val_loss: 0.2290
Epoch 872/3000
31920/31920 [==============================] - 1s 38us/step - loss: 4.8805e-06 - val_loss: 0.2290
Epoch 873/3000
31920/31920 [==============================] - 1s 38us/step - loss: 2.1904e-06 - val_loss: 0.2290
Epoch 874/3000
31920/31920 [==============================] - 1s 39us/step - loss: 4.1378e-06 - val_loss: 0.229

31920/31920 [==============================] - 1s 38us/step - loss: 4.4053e-06 - val_loss: 0.2286
Epoch 939/3000
31920/31920 [==============================] - 1s 39us/step - loss: 3.7702e-06 - val_loss: 0.2287
Epoch 940/3000
31920/31920 [==============================] - 1s 39us/step - loss: 3.2408e-06 - val_loss: 0.2287
Epoch 941/3000
31920/31920 [==============================] - 1s 38us/step - loss: 3.5849e-06 - val_loss: 0.2287
Epoch 942/3000
31920/31920 [==============================] - 1s 41us/step - loss: 3.4386e-06 - val_loss: 0.2286
Epoch 943/3000
31920/31920 [==============================] - 1s 39us/step - loss: 3.4020e-06 - val_loss: 0.2286
Epoch 944/3000
31920/31920 [==============================] - 1s 39us/step - loss: 2.8062e-06 - val_loss: 0.2287
Epoch 945/3000
31920/31920 [==============================] - 1s 38us/step - loss: 4.5121e-06 - val_loss: 0.2286
Epoch 946/3000
31920/31920 [==============================] - 1s 39us/step - loss: 3.9482e-06 - val_loss: 0.228

31920/31920 [==============================] - 1s 42us/step - loss: 5.0155e-06 - val_loss: 0.2283
Epoch 1011/3000
31920/31920 [==============================] - 1s 38us/step - loss: 1.0892e-06 - val_loss: 0.2283
Epoch 1012/3000
31920/31920 [==============================] - 1s 38us/step - loss: 4.9750e-06 - val_loss: 0.2282
Epoch 1013/3000
31920/31920 [==============================] - 1s 38us/step - loss: 2.3365e-06 - val_loss: 0.2283
Epoch 1014/3000
31920/31920 [==============================] - 1s 38us/step - loss: 2.8211e-06 - val_loss: 0.2283
Epoch 1015/3000
31920/31920 [==============================] - 1s 37us/step - loss: 3.5325e-06 - val_loss: 0.2283
Epoch 1016/3000
31920/31920 [==============================] - 1s 38us/step - loss: 2.3301e-06 - val_loss: 0.2282
Epoch 1017/3000
31920/31920 [==============================] - 1s 38us/step - loss: 6.1117e-06 - val_loss: 0.2282
Epoch 1018/3000
31920/31920 [==============================] - 1s 38us/step - loss: 4.8931e-07 - val_los

31920/31920 [==============================] - 1s 37us/step - loss: 4.8198e-06 - val_loss: 0.2279
Epoch 1083/3000
31920/31920 [==============================] - 1s 36us/step - loss: 1.2825e-06 - val_loss: 0.2279
Epoch 1084/3000
31920/31920 [==============================] - 1s 37us/step - loss: 4.4712e-06 - val_loss: 0.2280
Epoch 1085/3000
31920/31920 [==============================] - 6443s 202ms/step - loss: 1.0712e-06 - val_loss: 0.2279
Epoch 1086/3000
31920/31920 [==============================] - 3s 106us/step - loss: 2.8706e-06 - val_loss: 0.2279
Epoch 1087/3000
31920/31920 [==============================] - 3s 99us/step - loss: 3.6768e-06 - val_loss: 0.2279
Epoch 1088/3000
31920/31920 [==============================] - 3s 90us/step - loss: 4.5495e-06 - val_loss: 0.2279
Epoch 1089/3000
31920/31920 [==============================] - 3s 90us/step - loss: 4.9322e-06 - val_loss: 0.2279
Epoch 1090/3000
31920/31920 [==============================] - 3s 83us/step - loss: 8.6148e-07 - va

In [ ]:
model.save('my_model_3000.h5')

In [ ]:
embeddings = model.layers[2].get_weights()[0]

In [ ]:
embeddings.shape
# embed = embeddings.T
# print(embed.shape)

In [ ]:
name = "馬崇堯"
#name = "周O誼"

In [ ]:
def getCharEmbed(name):
    temp = []
    for char in name:
        try:
            temp.append(dict_char[char])
        except:
            k, v = random.choice(list(dict_char.items()))
            temp.append(v)
            
    if len(temp) == 2:
        user_emb = np.mean([temp[0], temp[1]], axis=0)
    elif len(temp) == 3:
        user_emb = np.mean([temp[0], temp[1], temp[2]], axis=0)
    elif len(temp) == 4:
        user_emb = np.mean([temp[0], temp[1], temp[2], temp[-1]], axis=0)
    else:
        k, v = random.choice(list(dict_char.items()))
        user_emb = v
    
    return user_emb

In [ ]:
u = getCharEmbed(name)
sc = np.array(range(32))

In [ ]:
print(u.shape)

In [ ]:
#model.predict([np.random.random((10, 256)), np.random.randint(0,32, (10))])
#np.random.random((10, 256))

In [ ]:
score_list = []
for i in range(32):
    output = model.predict([[u], [i]])
    if output[0][0] > 0.8:
        score_list.append(1)
    else:
        score_list.append(0)

In [ ]:
print(score_list)

In [ ]:
f = open ('./dataset/item_index_withEN.pkl', 'rb')
item_index_dict = pickle.load(f)
f.close()

item_index_dict 

In [ ]:
def getSchool(item_index_dict, score_list):
    broad_list = []
    for i in range(len(score_list)):
        if score_list[i] == 1:
            for k, v in item_index_dict.items():
                if v == i:
                    broad_list.append(k)
    return broad_list

In [ ]:
output_list = getSchool(item_index_dict, score_list)

In [ ]:
print(output_list)
print(len(output_list))